In [1]:
%pip install gradio --quiet


Note: you may need to restart the kernel to use updated packages.


In [2]:
import pickle
import json
import pprint
ml_model = pickle.load(open('ml_predictor.pkl', 'rb'))
with open("spec.json") as f:
    spec=json.load(f)

pprint.pprint(spec)

{'Air temperature (K)': {'dtype': 'float', 'max': 304.5, 'min': 295.3},
 'Process temperature (K)': {'dtype': 'float', 'max': 313.8, 'min': 305.7},
 'Rotational speed (rpm)': {'dtype': 'float', 'max': 2886.0, 'min': 1168.0},
 'Tool wear (min)': {'dtype': 'float', 'max': 253, 'min': 0},
 'Torque (Nm)': {'dtype': 'float', 'max': 76.6, 'min': 3.8},
 'Type': {'dtype': 'int', 'max': 2, 'min': 0}}


/opt/anaconda3/envs/ml/lib/python3.13/site-packages/sklearn/base.py:442: InconsistentVersionWarning: Trying to unpickle estimator DecisionTreeClassifier from version 1.6.1 when using version 1.7.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/opt/anaconda3/envs/ml/lib/python3.13/site-packages/sklearn/base.py:442: InconsistentVersionWarning: Trying to unpickle estimator RandomForestClassifier from version 1.6.1 when using version 1.7.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/opt/anaconda3/envs/ml/lib/python3.13/site-packages/sklearn/base.py:442: InconsistentVersionWarning: Trying to unpickle estimator MultiOutputClassifier from version 1.6.1 whe

In [ ]:
import numpy as np
import pandas as pd
import gradio as gr
headers=['Machine failure', 'TWF', 'HDF', 'PWF', 'OSF', 'RNF']
label_descriptions = {
    'Machine failure':'Machine failure',
    'TWF': 'Tool Wear Failure',
    'HDF': 'Heat Dissipation Failure',
    'PWF': 'Power Failure',
    'OSF': 'Overstrain Failure',
    'RNF': 'Random Failure'
}
machine_types={
    "Low":0,
    "Medium":1,
    "High":2
}
#['Type', 'Rotational speed (rpm)', 'Tool wear (min)', 'Strain (minNm)','Power (W)', 'Temperature Difference (K)'],
def predict(air_temperature, process_temperature, rotational_speed, torque, tool_wear, machine_type):
    df = pd.DataFrame(columns=['Type', 'Rotational speed (rpm)', 'Tool wear (min)', 'Strain (minNm)','Power (W)', 'Temperature Difference (K)'])
    if machine_type is None:
        machine_type = 'Low'
    machine_type_value = machine_types[machine_type]
    df['Type'] = [machine_type_value]
    df['Rotational speed (rpm)']=[rotational_speed]
    df['Tool wear (min)']=[tool_wear]
    df['Strain (minNm)'] = df['Tool wear (min)'] * torque 
    df['Power (W)'] =  df['Rotational speed (rpm)'] * torque * 2 * np.pi / 60
    df['Temperature Difference (K)'] = [process_temperature - air_temperature]

    y_probs=ml_model.predict_proba(df)
    thresholds = [0.5,0.5, 0.5,0.5,0.5,0.5]
    y_pred = np.column_stack([
        (y_probs[i][:, 1] > thresholds[i]).astype(int) for i in range(6)
    ])
    y_pred=pd.DataFrame(y_pred, columns=['Machine failure', 'TWF', 'HDF', 'PWF', 'OSF', 'RNF'])
    print(y_pred)
    y_pred['Machine failure']=np.where((y_pred['HDF']==1)|(y_pred['PWF']==1)|(y_pred['OSF']==1)|(y_pred['TWF']==1)|(y_pred['RNF']==1),1,0)
    failed = y_pred["Machine failure"].sum()>0
    result = ""
    maintenance = "No Action Needed"
    if failed:
        for c in y_pred.columns:
            value = y_pred[c][0]
            if value:
                result += f"\n - {label_descriptions[c]}"
            maintenance="Need Maintenance" 
    return result, maintenance


inputs=[]
for s, limits in spec.items():
    if s=='Type':
        continue
    max_s=limits["max"]
    min_s=limits["min"]
    inputs.append(gr.Slider(min_s, max_s, label=s)) 
inputs.append(gr.Radio(["Low", "Medium", "High"], label="Type", value="Low"))
demo = gr.Interface(
    title="ML Predictive Maintenance",
    fill_width=True,
    fn=predict,
    inputs=inputs,
    outputs=[
             # gr.Dataframe(datatype=["str","bool"], row_count=5, col_count=2, show_fullscreen_button=True),
             gr.components.Textbox(label="ML Analysis",lines=6), 
             gr.components.Textbox(label="Action")]
)
# Type,Air temperature (K),Process temperature (K),Rotational speed (rpm),Torque (Nm),Tool wear (min),Strain (minNm),Power (W),Temperature Difference (K)

# 0,304.4,313.7,1509,35.0,205,7175.0,5530.77386664483,9.300000000000011

demo.launch()

* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.


   Machine failure  TWF  HDF  PWF  OSF  RNF
0                1    0    1    1    1    0


- TWF fail if Tool Wear (Min) > 198
- OSF fail if Strain (minNm) >= 11019.2
- HDF Fail if Rotational Speed (rpm) <= 1379 and Temperature Difference <=8.599999999999966
- PWF fail if Power (W) <= 3521.78819652723 or Power (W) >= 8998.024014705741